In [ ]:
import numpy as np

class SDCAClassifier:
    def __init__(self, loss="squared", lam=1e-3, max_iter=1500, tol=1e-5, verbose=False):
        self.loss = loss
        self.lam = lam
        self.max_iter = max_iter
        self.tol = tol
        self.verbose = verbose
        self.w = None
        self.alpha = None
        self.losses_ = []

    def fit(self, X, y):
        n_samples, n_features = X.shape
        # Important: initialize alpha based on the loss requirements
        self.alpha = np.zeros(n_samples) 
        self.w = np.zeros(n_features)
        # random initialization 
        #self.alpha = np.random.randn(n_samples) * 0.01
        #using the formula of w in terms of alpha to initialize
        #self.w = (1 / (self.lam * n_samples)) * X.T @ self.alpha


        y_internal = y.copy()
        # Internal labels must be {-1, 1} for the dual conjugates used here
        if self.loss == "log":
            y_internal = np.where(y <= 0, -1, 1)

        
        for it in range(self.max_iter):
            indices = np.random.permutation(n_samples)
            for i in indices:
                xi = X[i]
                yi = y_internal[i]
                norm_sq = np.dot(xi, xi)
                if norm_sq == 0: continue

                pred = np.dot(self.w, xi)
                
                # Denominator for coordinate update
                # This represents the curvature of the objective
                denom = norm_sq / (self.lam * n_samples) + 1
                
                if self.loss == "squared":
                    # For squared loss, the optimal update is closed-form
                    delta = (yi - pred - self.alpha[i]) / denom
                
                elif self.loss == "log":
                    # For logistic, we use a Newton step
                    # Dual variable alpha_i for yi*u should be in [0, 1]
                    # Here we track alpha such that w = 1/(lam*n) sum alpha_i * xi
                    exp_term = np.exp(np.clip(yi * pred, -50, 50))
                    grad = yi / (1 + exp_term) - self.alpha[i]
                    delta = grad / denom
                    
                    # Projection: for logistic, yi * alpha_i must be in [0, 1]
                    # This is equivalent to alpha_i being between 0 and yi
                    new_alpha = np.clip(self.alpha[i] + delta, min(0, yi), max(0, yi))
                    delta = new_alpha - self.alpha[i]

                self.alpha[i] += delta
                # Incrementally update w to keep it in sync with alpha
                self.w += (delta / (self.lam * n_samples)) * xi
            
            # Recalculate metrics to check convergence
            primal_obj, dual_obj = self._compute_objectives(X, y_internal)
            gap = primal_obj - dual_obj
            self.losses_.append(primal_obj)

            if self.verbose and (it + 1) % 10 == 0:
                # We use Relative Gap to see progress better
                rel_gap = gap / max(abs(primal_obj), 1.0)
                #print(f"Iter {it+1:3d} | Primal: {primal_obj:.5f} | Gap: {gap:.2e} | RelGap: {rel_gap:.2%} | Loss: {self.loss}")
                print(f"Iter {it+1:3d} | Gap: {gap:.2e} | Primal Loss: {primal_obj:.6f}")
            
            if gap < self.tol:
                if self.verbose: print(f"Converged at iteration {it+1}")
                break
                
        return self

    def _compute_objectives(self, X, y):
        n = X.shape[0]
        preds = X @ self.w
        
        # 1. Primal Objective
        if self.loss == "squared":
            loss_term = np.mean(0.5 * (preds - y)**2)
        else:
            loss_term = np.mean(np.log(1 + np.exp(-y * preds)))
        primal_obj = loss_term + 0.5 * self.lam * np.dot(self.w, self.w)

        # 2. Dual Objective (Fenchel Dual)
        # Using the conjugate: phi*(v) = 0.5*v^2 + yi*v for squared
        # Using the conjugate: phi*(v) = v*log(v) + (1-v)*log(1-v) for logistic
        if self.loss == "squared":
            dual_res = np.mean(0.5 * self.alpha**2 - y * self.alpha)
        else:
            # For logistic, v = yi * alpha_i must be in [0, 1]
            v = np.clip(y * self.alpha, 1e-12, 1 - 1e-12)
            dual_res = np.mean(v * np.log(v) + (1 - v) * np.log(1 - v))
            
        dual_obj = -dual_res - 0.5 * self.lam * np.dot(self.w, self.w)
        return primal_obj, dual_obj

    def _compute_loss(self, X, y):
        n = X.shape[0]
        pred = X @ self.w
        if self.loss == "squared":
            loss = np.mean(0.5*(pred - y)**2) + 0.5 * self.lam * np.linalg.norm(self.w)**2
        elif self.loss == "log":
            z = y * pred
            loss = np.mean(np.log(1 + np.exp(-z))) + 0.5 * self.lam * np.linalg.norm(self.w)**2
        return loss
    
    def predict(self, X):
        return (X @ self.w > 0).astype(int)

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from ucimlrepo import fetch_ucirepo

# Fetch dataset
bank_marketing = fetch_ucirepo(id=222)
X = bank_marketing.data.features
y = (bank_marketing.data.targets == "yes").astype(int)  # convert yes/no -> 1/0

# One-hot encode categorical features
X = pd.get_dummies(X)

# Scale numeric features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y.values.ravel(), test_size=0.2, random_state=42
)

In [3]:
from sklearn.datasets import make_classification, make_regression
from sklearn.metrics import accuracy_score, mean_squared_error
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

lr_set = [1]
lams = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
lams.reverse()

best_loss = float('inf')
best_params = None

for lr in lr_set:
    for lam in lams:
        classifier = SDCAClassifier(loss="squared", lam=lam, verbose=0, tol=1e-6)
        classifier.fit(X_train, y_train)
        test_loss = classifier._compute_loss(X_test, y_test)
        y_pred = classifier.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"LR {lr:.1e} - Lambda {lam:.1e} - Test Loss: {test_loss:.6f}- Steps {len(classifier.losses_)} - Accuracy {accuracy}")
        
        if test_loss < best_loss:
            best_loss = test_loss
            best_params = (lr, lam) 

print("\nBest Parameters:")
print("LR:", best_params[0], "Lambda:", best_params[1], "Test Loss:", best_loss)

LR 1.0e+00 - Lambda 1.0e-01 - Test Loss: 0.044916- Steps 4 - Accuracy 0.753621585756939
LR 1.0e+00 - Lambda 1.0e-02 - Test Loss: 0.043791- Steps 18 - Accuracy 0.75494857901139
LR 1.0e+00 - Lambda 1.0e-03 - Test Loss: 0.043669- Steps 109 - Accuracy 0.7556120756386155
LR 1.0e+00 - Lambda 1.0e-04 - Test Loss: 0.043658- Steps 909 - Accuracy 0.7552803273250027


In [8]:
classifier = SDCAClassifier(loss="squared", lam=1e-3, max_iter=5000, verbose=1, tol=1e-5)
classifier.fit(X_train, y_train)

Iter  10 | Gap: 2.85e-04 | Primal Loss: 0.042853
Iter  20 | Gap: 1.48e-05 | Primal Loss: 0.042717
Converged at iteration 24


In [ ]:
plt.figure(figsize=(6,4))
plt.plot(classifier.losses_)
plt.xlabel("Iteration (every 10 steps)")
plt.ylabel("Training Loss")
plt.title("SCDA Training Loss Curve in Log Scale")
plt.yscale('log')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(classifier.losses_)
plt.xlabel("Iteration (every 10 steps)")
plt.ylabel("Training Loss")
plt.title("SCDA Training Loss Curve")
#plt.yscale('log')
plt.grid(True)
plt.show()

In [ ]:
classifier.losses_

In [9]:
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.7545062479265731


In [ ]:
test_loss = classifier._compute_loss(X_test, y_test)
print("Test Loss:", test_loss)